In [14]:
%matplotlib

Using matplotlib backend: TkAgg


In [15]:
import tkinter as tk
from pysrc.simplertplot.plotwidget import RTPlotWidget
from pysrc.simplertplot import queues
from hello.hello3 import HelloApp, TrueError
from time import time, sleep
import numpy as np
from matplotlib.ticker import NullFormatter, NullLocator, FormatStrFormatter

In [16]:
#ip = '192.168.1.85'
ip = '71.189.82.196:85'

In [17]:
interval = 1
minutes = .5
max_pts=minutes*60/(interval/1000)
max_pts=int(max_pts)
max_pts

30000

In [18]:
class MyPlotWidget(RTPlotWidget):
    def __init__(self, *args, **kw):
        super().__init__(*args, **kw)
        self._special()
    def _ma(self, a, n):
        rv = np.zeros(len(a))
        n //= 2
        if n < 1:
            n = 1
        for i in range(len(a)):
            rv[i] = np.mean(a[max(i-n, 0):i+n])
        return rv

    def _update_data(self):

#         _plot_x_data = self._ma(self.x_data.get(), 3)
#         _plot_y_data = self._ma(self.y_data.get(), 3)
#         _plot_y2_data = self._ma(self.y_data2.get(), 3)
        self.line.set_data(self.x_data.get(), self.y_data.get())
        self.line2.set_data(self.x_data.get(), self.y_data2.get())
        self.subplot.relim()
        self.subplot.autoscale_view(True, True, True)
        self.ax2.relim()
        self.ax2.autoscale_view(True,True,True)
        lower, upper = self.subplot.get_ybound()
        self.subplot.set_ylim(lower-0.2, upper+0.2, True, None)
        lower, upper = self.ax2.get_ybound()
        self.ax2.set_ylim(lower-1, upper+1, True, None)
        self._update()
        
    def _special(self):
        self.ax2 = self.subplot.twinx()
        self.y_data2 = queues.RingBuffer(self.max_pts)
        self.line2, = self.ax2.plot(self.x_data.get(), self.y_data2.get(), 'g-')
        self.yaxis2 = self.ax2.yaxis
        self.yaxis2.set_major_formatter(FormatStrFormatter("%.2f"))

    def extend_xy(self, x, y, y2):
        self.x_data.extend(x)
        self.y_data.extend(y)
        self.y_data2.extend(y2)
        self._update_data()
        
    def reset(self):
        self.x_data.clear()
        self.y_data.clear()
        self.y_data2.clear()
        self._update_data()

    def add_xy(self, x, y, y2):
        self.x_data.put(x)
        self.y_data.put(y)
        self.y_data2.put(y2)
        self._update_data()

In [19]:
def timer():
    start = time()
    while True:
        current = time()
        yield current - start
get_time = timer().__next__

In [20]:
g_stop_tasks = False
g_after = None
def tk_task(tk, interval, f):
    global g_after, g_stop_tasks
    def fn(*args, **kw):
        global g_after
        try:
            return f(*args, **kw)
        finally:
            if not g_stop_tasks:
                g_after = tk.after(interval, fn)
    g_after = tk.after(interval, fn)

In [21]:
import threading
class Model():
    def __init__(self, app, interval=0.1):
        self.mv = app.getMainValues()  # prepopulate
        self.lock = threading.RLock()
        self.running = threading.Event()
        self.thread = None
        self.app = app
        self.interval = interval
    def getvalues(self):
        with self.lock:
            pv = self.mv['agitation']['pv']
            op = self.mv['agitation']['output']
        return pv, op
    def getMainValues(self):
        # Use of a lock kind of useless since a)
        # no copy of the data is made and b)
        # self.mv is assigned a brand new dict
        # object with each update. 
        with self.lock:
            mv = self.mv
        return mv
    def run_update(self):
        self.running.set()
        self.thread = threading.Thread(None, self._run_update, daemon=True)
        self.thread.start()
    def _run_update(self):
        while self.running.is_set():
            try:
                mv = self.app.getMainValues()
            except Exception:
                continue
            with self.lock:
                self.mv = mv
            sleep(self.interval)
    def stop(self):
        self.running.clear()

h = HelloApp(ip)
m = Model(h, 0)
m.run_update()

def update():
    # Note to future self yes this is stupid
    # no i don't care (right now)
    with m.lock:
        mv = m.getMainValues()
        ag = mv['agitation']
        pv = ag['pv']
        op = ag['output']
    t = get_time()
    p.add_xy(t, pv, op)
    update_labels(pv, op)

In [9]:
def printdir(o):
    for a in dir(o):
        if not a.startswith("_"):
            print(a, getattr(o, a))

In [10]:
def conf2(ev):
    if ev.widget == root:
        p.tkcanvas.config(height=ev.height, width=ev.width-rbtn.winfo_width())

In [11]:
root = tk.Tk()
root.withdraw()

p = MyPlotWidget(root, max_pts=max_pts)
p.tkcanvas.config(height=400, width=600)
max_pts


30000

In [12]:
class ILabel(tk.Label):
    def __init__(self, master, name, value=0, fmt="%.2f"):
        super().__init__(master)
        self.fmt = fmt
        self.name = name
        self.set_value(value)
        
    def set_value(self, val):
        val = self.fmt % val
        self.config(text="%s: %s" % (self.name, val))
   
labels = {}
def mk_label(name, val, row):
    l = ILabel(root, name, val)
    labels[name] = l
    l.grid(row=row, column=1)
    
mk_label("PV", 0, 1)
mk_label("OP", 0, 2)
    
def update_labels(pv, op):
    labels['PV'].set_value(pv)
    labels['OP'].set_value(op)

    

In [13]:
p.grid(rowspan=20, column=0, row=0)    
def reset():
    p.reset()
    global get_time
    get_time = timer().__next__
rbtn = tk.Button(root, text="Reset", command=reset)
rbtn.grid(column=1, row=0, sticky="ew")
root.bind("<Configure>", conf2)
p.set_xaxis_text("Time (s)")
p.set_yaxis_text("DO PV (%)")
def on_closing():
    from tkinter import messagebox
    if messagebox.askokcancel("Quit", "Do you want to quit?"):
        global g_stop_tasks, g_after
        g_stop_tasks = True
        root.after_cancel(g_after)
        root.quit()
        root.destroy()

root.protocol("WM_DELETE_WINDOW", on_closing)
root.wm_deiconify()
tk_task(root, interval, update)
try:
    root.mainloop()
finally:
    m.stop()
